In [1]:
import pandas as pd
import geopandas as gpd
import h3
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Point
from shapely.geometry import Polygon

In [7]:
def points_to_grid (points_gdf, edge_size):
    square_gdf = points_gdf.copy()
    for index, row in square_gdf.iterrows():
        point = row['geometry']
        
        # Define the coordinates for the corners of the square
        x, y = point.x, point.y
        square_half_size = edge_size*0.5
        
        # Create the coordinates for the square's corners
        square_coords = [(x - square_half_size, y - square_half_size),
                        (x + square_half_size, y - square_half_size),
                        (x + square_half_size, y + square_half_size),
                        (x - square_half_size, y + square_half_size)]
        
        polygon = Polygon(square_coords)
        square_gdf.loc[index, 'geometry'] = polygon
    return square_gdf

In [2]:
cowpea = pd.read_csv('cowpea_gridcell_summary.csv')
cowpea_gdf = gpd.GeoDataFrame(cowpea, geometry=gpd.points_from_xy(cowpea.long, cowpea.lat))
cowpea_gdf.set_crs(epsg=4326, inplace=True)
cowpea_gdf

,exp,long,lat,baseline_yield,ssp126_gfdl_av,ssp370_gfdl_av,ssp126_ipsl_av,ssp370_ipsl_av,ssp126_gfdl_pc,ssp370_gfdl_pc,ssp126_ipsl_pc,ssp370_ipsl_pc,geometry
0,114115,-2.75,10.75,1885,1842,1411,1943,1663,-2.28,-25.15,3.08,-11.78,POINT (-2.75000 10.75000)
1,114115,-2.75,10.75,1767,1979,1775,1701,1683,12.00,0.45,-3.74,-4.75,POINT (-2.75000 10.75000)
2,114115,-2.75,10.75,1742,1823,1870,1767,1540,4.65,7.35,1.44,-11.60,POINT (-2.75000 10.75000)
3,114115,-2.75,10.75,1766,1870,1881,1381,1392,5.89,6.51,-21.80,-21.18,POINT (-2.75000 10.75000)
4,114115,-2.75,10.75,1834,1997,1902,1588,1786,8.89,3.71,-13.41,-2.62,POINT (-2.75000 10.75000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,120601,0.25,6.25,1557,745,1626,1580,1141,-52.15,4.43,1.48,-26.72,POINT (0.25000 6.25000)
1016,120601,0.25,6.25,1625,1690,1542,1618,1341,4.00,-5.11,-0.43,-17.48,POINT (0.25000 6.25000)
1017,120601,0.25,6.25,1486,1668,1641,1307,1032,12.25,10.43,-12.05,-30.55,POINT (0.25000 6.25000)
1018,120601,0.25,6.25,1621,1760,1567,1513,1150,8.57,-3.33,-6.66,-29.06,POINT (0.25000 6.25000)


In [31]:

average_cowpea = cowpea.groupby('exp').mean()
average_cowpea["value_type"] = "mean"
average_cowpea = average_cowpea.reset_index()

max_cowpea = cowpea.groupby('exp').max()
max_cowpea["value_type"] = "max"
max_cowpea = max_cowpea.reset_index()

min_cowpea = cowpea.groupby('exp').min()
min_cowpea["value_type"] = "min"
min_cowpea = min_cowpea.reset_index()

cowpea_summary = pd.concat([average_cowpea, max_cowpea, min_cowpea])

cowpea_summary_gdf_point = gpd.GeoDataFrame(cowpea_summary, geometry=gpd.points_from_xy(cowpea_summary.long, cowpea_summary.lat))
cowpea_summary_gdf_point.set_crs(epsg=4326, inplace=True)


,exp,long,lat,baseline_yield,ssp126_gfdl_av,ssp370_gfdl_av,ssp126_ipsl_av,ssp370_ipsl_av,ssp126_gfdl_pc,ssp370_gfdl_pc,ssp126_ipsl_pc,ssp370_ipsl_pc,value_type,geometry
0,114115,-2.75,10.75,1770.00,1858.80,1756.05,1614.65,1386.50,5.125,-0.6590,-8.7290,-21.6865,mean,POINT (-2.75000 10.75000)
1,114116,-2.25,10.75,1762.95,1845.15,1771.40,1638.95,1419.50,4.762,0.5400,-6.9335,-19.3730,mean,POINT (-2.25000 10.75000)
2,114117,-1.75,10.75,1719.05,1767.20,1689.50,1520.05,1285.55,2.971,-1.5430,-11.5285,-25.1335,mean,POINT (-1.75000 10.75000)
3,114118,-1.25,10.75,1686.70,1685.60,1558.60,1422.35,1156.70,0.077,-7.2415,-15.4780,-31.2530,mean,POINT (-1.25000 10.75000)
4,114119,-0.75,10.75,1693.70,1709.60,1619.15,1450.35,1189.20,1.090,-4.0335,-14.2590,-29.6895,mean,POINT (-0.75000 10.75000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,119880,-0.25,6.75,1532.00,778.00,1320.00,1086.00,855.00,-50.730,-19.1700,-37.0400,-47.3500,min,POINT (-0.25000 6.75000)
47,119881,0.25,6.75,1608.00,1040.00,1543.00,1216.00,1008.00,-37.160,-6.9100,-28.8500,-37.3100,min,POINT (0.25000 6.75000)
48,120599,-0.75,6.25,1408.00,932.00,1265.00,1061.00,985.00,-36.430,-15.6700,-29.2700,-33.7600,min,POINT (-0.75000 6.25000)
49,120600,-0.25,6.25,1569.00,1253.00,1578.00,1358.00,1219.00,-22.700,-5.5000,-18.8300,-27.2300,min,POINT (-0.25000 6.25000)


In [32]:
cowpea_summary_gdf_point

,exp,long,lat,baseline_yield,ssp126_gfdl_av,ssp370_gfdl_av,ssp126_ipsl_av,ssp370_ipsl_av,ssp126_gfdl_pc,ssp370_gfdl_pc,ssp126_ipsl_pc,ssp370_ipsl_pc,value_type,geometry
0,114115,-2.75,10.75,1770.00,1858.80,1756.05,1614.65,1386.50,5.125,-0.6590,-8.7290,-21.6865,mean,POINT (-2.75000 10.75000)
1,114116,-2.25,10.75,1762.95,1845.15,1771.40,1638.95,1419.50,4.762,0.5400,-6.9335,-19.3730,mean,POINT (-2.25000 10.75000)
2,114117,-1.75,10.75,1719.05,1767.20,1689.50,1520.05,1285.55,2.971,-1.5430,-11.5285,-25.1335,mean,POINT (-1.75000 10.75000)
3,114118,-1.25,10.75,1686.70,1685.60,1558.60,1422.35,1156.70,0.077,-7.2415,-15.4780,-31.2530,mean,POINT (-1.25000 10.75000)
4,114119,-0.75,10.75,1693.70,1709.60,1619.15,1450.35,1189.20,1.090,-4.0335,-14.2590,-29.6895,mean,POINT (-0.75000 10.75000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,119880,-0.25,6.75,1532.00,778.00,1320.00,1086.00,855.00,-50.730,-19.1700,-37.0400,-47.3500,min,POINT (-0.25000 6.75000)
47,119881,0.25,6.75,1608.00,1040.00,1543.00,1216.00,1008.00,-37.160,-6.9100,-28.8500,-37.3100,min,POINT (0.25000 6.75000)
48,120599,-0.75,6.25,1408.00,932.00,1265.00,1061.00,985.00,-36.430,-15.6700,-29.2700,-33.7600,min,POINT (-0.75000 6.25000)
49,120600,-0.25,6.25,1569.00,1253.00,1578.00,1358.00,1219.00,-22.700,-5.5000,-18.8300,-27.2300,min,POINT (-0.25000 6.25000)


In [33]:
cowpea_summary_gdf_poly = points_to_grid (cowpea_summary_gdf_point, 0.5)
cowpea_summary_gdf = pd.concat([cowpea_summary_gdf_point, cowpea_summmary_gdf_poly])
cowpea_summary_gdf.to_file("cowpea_summary.geojson", driver='GeoJSON')

In [35]:
cowpea_summary_gdf.describe()

,exp,long,lat,baseline_yield,ssp126_gfdl_av,ssp370_gfdl_av,ssp126_ipsl_av,ssp370_ipsl_av,ssp126_gfdl_pc,ssp370_gfdl_pc,ssp126_ipsl_pc,ssp370_ipsl_pc
count,1173.000000,1173.000000,1173.000000,1173.000000,1173.000000,1173.000000,1173.000000,1173.000000,1173.000000,1173.000000,1173.000000,1173.000000
mean,116984.098039,-1.142157,8.759804,1576.905286,1530.414706,1515.327536,1437.721398,1180.646249,-2.889749,-4.010267,-8.459831,-25.384838
std,2040.082622,0.920291,1.416516,183.737788,342.316945,312.434419,277.695549,396.798975,21.255480,17.883103,17.848989,24.216443
min,114115.000000,-2.750000,6.250000,665.000000,282.000000,250.000000,328.000000,113.000000,-69.580000,-79.690000,-71.380000,-92.100000
25%,114841.000000,-1.750000,7.250000,1523.000000,1504.000000,1458.000000,1331.000000,971.000000,-7.610000,-7.090000,-15.490000,-37.470000
50%,116996.000000,-1.250000,8.750000,1604.000000,1644.000000,1606.000000,1507.000000,1320.000000,1.510000,0.000000,-5.610000,-18.590000
75%,119156.000000,-0.250000,10.250000,1681.000000,1725.000000,1702.000000,1620.000000,1470.000000,6.580000,4.790000,0.180000,-9.120000
max,120601.000000,0.250000,10.750000,1971.000000,2051.000000,1977.000000,1943.000000,1786.000000,133.980000,64.440000,80.920000,64.430000


In [36]:
cowpea_summary_gdf

,exp,long,lat,baseline_yield,ssp126_gfdl_av,ssp370_gfdl_av,ssp126_ipsl_av,ssp370_ipsl_av,ssp126_gfdl_pc,ssp370_gfdl_pc,ssp126_ipsl_pc,ssp370_ipsl_pc,value_type,geometry
0,114115,-2.75,10.75,1770.00,1858.80,1756.05,1614.65,1386.50,5.125,-0.6590,-8.7290,-21.6865,mean,POINT (-2.75000 10.75000)
1,114116,-2.25,10.75,1762.95,1845.15,1771.40,1638.95,1419.50,4.762,0.5400,-6.9335,-19.3730,mean,POINT (-2.25000 10.75000)
2,114117,-1.75,10.75,1719.05,1767.20,1689.50,1520.05,1285.55,2.971,-1.5430,-11.5285,-25.1335,mean,POINT (-1.75000 10.75000)
3,114118,-1.25,10.75,1686.70,1685.60,1558.60,1422.35,1156.70,0.077,-7.2415,-15.4780,-31.2530,mean,POINT (-1.25000 10.75000)
4,114119,-0.75,10.75,1693.70,1709.60,1619.15,1450.35,1189.20,1.090,-4.0335,-14.2590,-29.6895,mean,POINT (-0.75000 10.75000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,120601,0.25,6.25,1557.00,745.00,1626.00,1580.00,1141.00,-52.150,4.4300,1.4800,-26.7200,NaN,"POLYGON ((0.00000 6.00000, 0.50000 6.00000, 0...."
1016,120601,0.25,6.25,1625.00,1690.00,1542.00,1618.00,1341.00,4.000,-5.1100,-0.4300,-17.4800,NaN,"POLYGON ((0.00000 6.00000, 0.50000 6.00000, 0...."
1017,120601,0.25,6.25,1486.00,1668.00,1641.00,1307.00,1032.00,12.250,10.4300,-12.0500,-30.5500,NaN,"POLYGON ((0.00000 6.00000, 0.50000 6.00000, 0...."
1018,120601,0.25,6.25,1621.00,1760.00,1567.00,1513.00,1150.00,8.570,-3.3300,-6.6600,-29.0600,NaN,"POLYGON ((0.00000 6.00000, 0.50000 6.00000, 0...."


# Past work below

In [29]:
def points_to_grid (points_gdf, edge_size):
    square_gdf = points_gdf.copy()
    for index, row in square_gdf.iterrows():
        point = row['geometry']
        
        # Define the coordinates for the corners of the square
        x, y = point.x, point.y
        square_half_size = edge_size*0.5
        
        # Create the coordinates for the square's corners
        square_coords = [(x - square_half_size, y - square_half_size),
                        (x + square_half_size, y - square_half_size),
                        (x + square_half_size, y + square_half_size),
                        (x - square_half_size, y + square_half_size)]
        
        polygon = Polygon(square_coords)
        square_gdf.loc[index, 'geometry'] = polygon
    return square_gdf

In [84]:
# write tester to geojson
tester.to_file("cowpea_gridcell_summary_poly.geojson", driver='GeoJSON')


In [64]:

polygons = []
attributes = []

# Iterate through each point in the original GeoDataFrame
for index, row in tester.iterrows():
    point = row['geometry']  # Get the point geometry
    
    # Define the coordinates for the corners of the square
    x, y = point.x, point.y
    square_half_size = 0.25  # Half of 0.25 degrees
    
    # Create the coordinates for the square's corners
    square_coords = [(x - square_half_size, y - square_half_size),
                     (x + square_half_size, y - square_half_size),
                     (x + square_half_size, y + square_half_size),
                     (x - square_half_size, y + square_half_size)]
    
    # Create a Polygon geometry from the coordinates
    polygon = Polygon(square_coords)
    polygons.append(polygon)

# Create a new GeoDataFrame with the polygons
squares_gdf = gpd.GeoDataFrame(geometry=polygons, crs=tester.crs)
# replace tester geometry with squares_gdf geometry
tester['geometry'] = squares_gdf['geometry']

In [65]:
squares_gdf

,long,lat,baseline_yield,ssp126_gfdl_av,ssp370_gfdl_av,ssp126_ipsl_av,ssp370_ipsl_av,ssp126_gfdl_pc,ssp370_gfdl_pc,ssp126_ipsl_pc,ssp370_ipsl_pc,geometry
exp,,,,,,,,,,,,
114115,-2.75,10.75,1770.00,1858.80,1756.05,1614.65,1386.50,5.1250,-0.6590,-8.7290,-21.6865,None
114116,-2.25,10.75,1762.95,1845.15,1771.40,1638.95,1419.50,4.7620,0.5400,-6.9335,-19.3730,None
114117,-1.75,10.75,1719.05,1767.20,1689.50,1520.05,1285.55,2.9710,-1.5430,-11.5285,-25.1335,None
114118,-1.25,10.75,1686.70,1685.60,1558.60,1422.35,1156.70,0.0770,-7.2415,-15.4780,-31.2530,None
114119,-0.75,10.75,1693.70,1709.60,1619.15,1450.35,1189.20,1.0900,-4.0335,-14.2590,-29.6895,None
114120,-0.25,10.75,1696.80,1708.70,1623.95,1464.20,1222.30,0.8500,-4.0005,-13.6920,-27.8520,None
114835,-2.75,10.25,1751.15,1822.45,1745.65,1610.60,1397.60,4.3245,-0.0975,-7.9195,-20.0025,None
114836,-2.25,10.25,1738.20,1825.70,1785.20,1646.45,1446.90,5.1835,2.7500,-5.1825,-16.6935,None
114837,-1.75,10.25,1711.45,1773.90,1704.90,1551.30,1328.65,3.8235,-0.3015,-9.2885,-22.3160,None


In [8]:
print("Max: ", cowpea['lat'].max())
print("Min: ", cowpea['lat'].min())
print("Max: ", cowpea['long'].max())
print("Min: ", cowpea['long'].min())

Max:  10.75
Min:  6.25
Max:  0.25
Min:  -2.75


In [ ]:
# for reference
lat, lng = 37.769377, -122.388903
resolution = 9
h3.latlng_to_cell(lat, lng, resolution)